<a href="https://colab.research.google.com/github/code-sanya/UCLA_Technology_Studio/blob/main/Sanya_AUD_ESRGAN_upRESer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AUD.IDEAS.TECHNOLOGY.STUDIO upRES-er NOTEBOOK
Use this colab to increase the resolution of images ran in VQGANCLIP or StyleTransfer from Studio     

This coLab is based off of the original "ESRGAN (new arch.) on Colab" model found here:  https://github.com/xinntao/ESRGAN and curated by M. Ahabb (Ahabbscience Studio)     

**IMPORTANT:** Read all text before running the cell as some directions can change parameters or might download to the incorrect folder 
Read the output of each cell after you run it > if you get an error or something didnt load or it cant find a folder or image then SOMETHING IS WRONG.

#**PART 01 | THE SETUP**

In [ ]:
#@title **STEP 00:**  Mount to your Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title **STEP 01:** Change Directory + Download ESRGAN
#@markdown you need to first create a folder named "upRes-main" in your Google Drive    

#@markdown If you have already cloned this Repo then you will get a fatal notice saying it cannot download because it already exists.
%cd //content/drive/MyDrive/upRes-main
!git clone https://github.com/xinntao/ESRGAN

/content/drive/MyDrive/upRes-main
fatal: destination path 'ESRGAN' already exists and is not an empty directory.


#**PART 02 | UPLOAD IMAGES + DOWNLOAD MODELS** 

In [ ]:
#@title **STEP 02:** Upload images from your computer
#@markdown Remember to delete any old images in the LR folder before this step
%cd /content/drive/MyDrive/upRes-main/ESRGAN/LR

from google.colab import files
uploaded = files.upload() 
for filename in uploaded.keys():
  print('User uploaded file "{name}" with {length} bytes'.format(name=filename, length=len(uploaded[filename])))

/content/drive/MyDrive/upRes-main/ESRGAN/LR


Saving 0001.png to 0001 (1).png
Saving 0002.png to 0002 (1).png
Saving 0003.png to 0003 (1).png
Saving 0004.png to 0004 (1).png
Saving 0005.png to 0005 (1).png
Saving 0006.png to 0006 (1).png
Saving 0007.png to 0007 (1).png
Saving 0008.png to 0008 (1).png
Saving 0009.png to 0009 (1).png
Saving 0010.png to 0010 (1).png
Saving 0011.png to 0011 (1).png
Saving 0012.png to 0012 (1).png
Saving 0013.png to 0013 (1).png
Saving 0014.png to 0014 (1).png
Saving 0015.png to 0015 (1).png
Saving 0016.png to 0016 (1).png
Saving 0017.png to 0017 (1).png
Saving 0018.png to 0018 (1).png
Saving 0019.png to 0019 (1).png
Saving 0020.png to 0020 (1).png
Saving 0021.png to 0021 (1).png
Saving 0022.png to 0022 (1).png
Saving 0023.png to 0023 (1).png
Saving 0024.png to 0024 (1).png
Saving 0025.png to 0025 (1).png
Saving 0026.png to 0026 (1).png
Saving 0027.png to 0027 (1).png
Saving 0028.png to 0028 (1).png
Saving 0029.png to 0029 (1).png
Saving 0030.png to 0030 (1).png
Saving 0031.png to 0031 (1).png
Saving 0

In [ ]:
#@title **STEP 03:** Download pretrained upResing models
%cd /content/drive/MyDrive/upRes-main/ESRGAN
import gdown
print("Downloading pretrained models")
output1 = '/content/drive/MyDrive/upRes-main/ESRGAN/models/RRDB_ESRGAN_x4.pth'
output2 = '/content/drive/MyDrive/upRes-main/ESRGAN/models/RRDB_PSNR_x4.pth'
output3 = '/content/drive/MyDrive/upRes-main/ESRGAN/models/PPON_D.pth'
output4 = '/content/drive/MyDrive/upRes-main/ESRGAN/models/PPON_G.pth'
print ('Downloading RRDB_ESRGAN_x4.pth')
gdown.download('https://drive.google.com/uc?id=1TPrz5QKd8DHHt1k8SRtm6tMiPjz_Qene', output1, quiet=True) 
print ('Downloading RRDB_PSNR_x4.pth')
gdown.download('https://drive.google.com/uc?id=1pJ_T-V1dpb1ewoEra1TGSWl5e6H7M4NN', output2, quiet=True)
print ('Downloading PPON_D.pth by Zheng Hui')
gdown.download('https://drive.google.com/uc?id=1Fr5aKCD6mw6P-hI0BZr6My2gHNhtUk-V', output3, quiet=True) 
print ('Downloading PPON_G.pth by Zheng Hui')
gdown.download('https://drive.google.com/uc?id=12uR3BSftNA0HDYiKda23GyAj_crpSjOm', output4, quiet=True)  

/content/drive/MyDrive/upRes-main/ESRGAN


'/content/drive/MyDrive/upRes-main/ESRGAN/models/PPON_G.pth'

#**PART 03 | CHOOSE A MODEL AND UPRES IT** 

In [ ]:
#@title **STEP 04:** Upscale the images / frames 
%cd /content/drive/MyDrive/upRes-main/ESRGAN

import os.path as osp
import glob
import cv2
import numpy as np
import torch
import RRDBNet_arch as arch
import requests
import imageio
import requests
import warnings
warnings.filterwarnings("ignore")
from google.colab import files

Choose_device = "cuda"  #@param ["cuda","cpu"]

model_path = 'models/RRDB_ESRGAN_x4.pth' #@param ['models/RRDB_ESRGAN_x4.pth','models/RRDB_PSNR_x4.pth']  
device = torch.device(Choose_device) 


test_img_folder = 'LR/*'

model = arch.RRDBNet(3, 3, 64, 23, gc=32)
model.load_state_dict(torch.load(model_path), strict=True)
model.eval()
model = model.to(device)

print('Model path {:s}. \nTesting...'.format(model_path))

idx = 0
for path in glob.glob(test_img_folder):
    idx += 1
    base = osp.splitext(osp.basename(path))[0]
    print(idx, base)
    # read images
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = img * 1.0 / 255
    img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1))).float()
    img_LR = img.unsqueeze(0)
    img_LR = img_LR.to(device)

    with torch.no_grad():
        output = model(img_LR).data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
    output = (output * 255.0).round()
    cv2.imwrite('results/{:s}.png'.format(base), output)

/content/drive/MyDrive/upRes-main/ESRGAN
Model path models/RRDB_ESRGAN_x4.pth. 
Testing...
1 0001
2 0002
3 0003
4 0004
5 0005
6 0006
7 0007
8 0008
9 0009
10 0010
11 0011
12 0012
13 0013
14 0014
15 0015
16 0016
17 0017
18 0018
19 0019
20 0020
21 0021
22 0022
23 0023
24 0024
25 0025
26 0026
27 0027
28 0028
29 0029
30 0030
31 0031
32 0032
33 0033
34 0034
35 0035
36 0036
37 0037
38 0038
39 0039
40 0040
41 0041
42 0042
43 0043
44 0044
45 0045
46 0046
47 0047
48 0048
49 0049
50 0050
51 0051
52 0052
53 0053
54 0054
55 0055
56 0056
57 0057
58 0058
59 0059
60 0060
61 0061
62 0062
63 0063
64 0064
65 0065
66 0066
67 0067
68 0068
69 0069
70 0070
71 0071
72 0072
73 0073
74 0074
75 0075
76 0076
77 0077
78 0078
79 0079
80 0080
81 0081
82 0082
83 0083
84 0084
85 0085
86 0086
87 0087
88 0088
89 0089
90 0090
91 0091
92 0092
93 0093
94 0094
95 0095
96 0096
97 0097
98 0098
99 0099
100 0100
101 0101
102 0102
103 0103
104 0104
105 0105
106 0106
107 0107
108 0108
109 0109
110 0110
111 0111
112 0112
113 0113


In [ ]:
#@title **STEP 05:** Make a ZIP file of the results

!zip -r results.zip /content/drive/MyDrive/upRes-main/ESRGAN/results

updating: content/drive/MyDrive/upRes-main/ESRGAN/results/ (stored 0%)
updating: content/drive/MyDrive/upRes-main/ESRGAN/results/baboon_ESRGAN.png (deflated 0%)
updating: content/drive/MyDrive/upRes-main/ESRGAN/results/baboon.png (deflated 0%)
updating: content/drive/MyDrive/upRes-main/ESRGAN/results/comic.png (deflated 0%)
updating: content/drive/MyDrive/upRes-main/ESRGAN/results/0142.png (deflated 0%)
updating: content/drive/MyDrive/upRes-main/ESRGAN/results/0143.png (deflated 0%)
updating: content/drive/MyDrive/upRes-main/ESRGAN/results/0144.png (deflated 0%)
updating: content/drive/MyDrive/upRes-main/ESRGAN/results/0001.png (deflated 0%)
updating: content/drive/MyDrive/upRes-main/ESRGAN/results/0002.png (deflated 1%)
updating: content/drive/MyDrive/upRes-main/ESRGAN/results/0003.png (deflated 0%)
updating: content/drive/MyDrive/upRes-main/ESRGAN/results/0004.png (deflated 1%)
updating: content/drive/MyDrive/upRes-main/ESRGAN/results/0005.png (deflated 0%)
updating: content/drive/My

In [ ]:
#@title **STEP 06:** Download the ZIP file
#@markdown You can also just manually download the results folder
files.download('results.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

This notebook was put together by Laure Michelon at https://github.com/laure-m